<a href="https://colab.research.google.com/github/Namanbansal9414/User-Referral-Rewards/blob/main/Project%20_Translator_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import pickle

In [27]:
class LanguageModel:
    def __init__(self):
        # Placeholders for sub-models and tools
        self.language_detection_model = None
        self.language_detection_cv = None

        self.french_translation_model = None
        self.french_cv = None
        self.french_label_encoder = None

        self.urdu_translation_model = None
        self.urdu_cv = None
        self.urdu_label_encoder = None

    def train_language_detection(self, path="/content/dataset.csv"):
        data = pd.read_csv(path)
        X = np.array(data["Text"])
        y = np.array(data["language"])
        self.language_detection_cv = CountVectorizer()
        X_vec = self.language_detection_cv.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.33, random_state=42)
        self.language_detection_model = MultinomialNB()
        self.language_detection_model.fit(X_train, y_train)
        acc = self.language_detection_model.score(X_test, y_test)
        print(f"Language Detection Accuracy: {acc:.4f}")

    def train_french_translation(self, path="/content/data1.csv"):
        data = pd.read_csv(path)
        X = np.array(data['English'])
        y = np.array(data['French'])
        self.french_label_encoder = LabelEncoder()
        y_encoded = self.french_label_encoder.fit_transform(y)
        X_train, _, y_train, _ = train_test_split(X, y_encoded, test_size=0.99, random_state=42)
        self.french_cv = CountVectorizer()
        X_train_vec = self.french_cv.fit_transform(X_train)
        self.french_translation_model = MultinomialNB()
        self.french_translation_model.fit(X_train_vec, y_train)
        acc = self.french_translation_model.score(X_train_vec, y_train)
        print(f"English to French Translation Accuracy: {acc:.4f}")

    def train_urdu_translation(self, path="/content/EU.xlsx"):
        data = pd.read_excel(path)
        X = np.array(data['English'])
        y = np.array(data['Urdu'])
        self.urdu_label_encoder = LabelEncoder()
        y_encoded = self.urdu_label_encoder.fit_transform(y)
        X_train, _, y_train, _ = train_test_split(X, y_encoded, test_size=0.42, random_state=42)
        self.urdu_cv = CountVectorizer()
        X_train_vec = self.urdu_cv.fit_transform(X_train)
        self.urdu_translation_model = MultinomialNB()
        self.urdu_translation_model.fit(X_train_vec, y_train)
        acc = self.urdu_translation_model.score(X_train_vec, y_train)
        print(f"English to Urdu Translation Accuracy: {acc:.4f}")

    def detect_language(self, text):
        x_vec = self.language_detection_cv.transform([text])
        pred = self.language_detection_model.predict(x_vec)[0]
        return pred

    def translate(self, text, target_language):
        if target_language.lower() == 'french':
            x_vec = self.french_cv.transform([text])
            pred_label = self.french_translation_model.predict(x_vec)[0]
            return self.french_label_encoder.inverse_transform([pred_label])[0]
        elif target_language.lower() == 'urdu':
            x_vec = self.urdu_cv.transform([text])
            pred_label = self.urdu_translation_model.predict(x_vec)[0]
            return self.urdu_label_encoder.inverse_transform([pred_label])[0]
        else:
            raise ValueError(f"Unsupported target language: {target_language}")

In [28]:
model = LanguageModel()

print("Training language detection model...")
model.train_language_detection()

print("Training English to French translation model...")
model.train_french_translation()

print("Training English to Urdu translation model...")
model.train_urdu_translation()

# Save the entire combined model to a pickle file
with open("/content/combined_language_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("Combined model saved as combined_language_model.pkl")

Training language detection model...
Language Detection Accuracy: 0.9532
Training English to French translation model...
English to French Translation Accuracy: 0.9818
Training English to Urdu translation model...
English to Urdu Translation Accuracy: 0.2782
Combined model saved as combined_language_model.pkl


In [29]:
!pip install gradio -q
import gradio as gr
import pickle

with open("/content/combined_language_model.pkl", "rb") as f:
    language_model = pickle.load(f)


def process_input(user_text, task):
    if not user_text.strip():
        return "⚠️ Please enter some text."
    if task == "Detect Language":
        return f"🌐 **Detected Language:** `{language_model.detect_language(user_text)}`"
    elif task == "Translate to French":
        return f"🇫🇷 **French Translation:**\n\n`{language_model.translate(user_text, 'french')}`"
    elif task == "Translate to Urdu":
        return f"🇵🇰 **Urdu Translation:**\n\n`{language_model.translate(user_text, 'urdu')}`"
    else:
        return "❌ Invalid task selected."

# Enhanced HTML with modern styling
description_html = """
<div style="text-align: center; font-family: 'Segoe UI', sans-serif; margin-bottom: 20px;">
    <h1 style="color: #1e88e5; font-size: 2.8em;">🌍 Multilingual AI Assistant</h1>
    <p style="font-size: 1.2em; color: #444;">
        Easily detect language 🌐 or translate English to <b style="color:#1e88e5;">French</b> 🇫🇷 or <b style="color:#388e3c;">Urdu</b> 🇵🇰
    </p>
</div>
"""

footer_html = """
<div style="text-align: center; margin-top: 40px; color: #888; font-size: 0.9em; font-family: 'Segoe UI', sans-serif;">
    <hr style="margin-bottom: 15px;">
    <p>Built with ❤️ using Python, Scikit-learn, and Gradio</p>
</div>
"""

# Interface with enhanced style
interface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Textbox(lines=3, placeholder="Type or paste your text here...", label="✏️ Enter Text"),
        gr.Dropdown(
            choices=["Detect Language", "Translate to French", "Translate to Urdu"],
            label="🧭 Select Action",
            value="Detect Language"
        ),
    ],
    outputs=gr.Markdown(label="🔍 Result"),
    description=description_html,
    article=footer_html,
    theme="default",
    allow_flagging="never"
)

interface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d2bfe1c6851a2163b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
